In [ ]:
from src.guitar.Guitar import Guitar
from src.hand.LeftFinger import LeftFinger
from src.hand.LeftHand import LeftHand
from src.guitar.GuitarString import createGuitarStrings
from src.midi.midiToNote import midiToGuitarNotes, get_tempo_changes,calculate_frame
from src.HandPoseRecorder import HandPoseRecorder,HandPoseRecordPool

## Import Midi file 导入Midi文件
Import midi file and convert it to a list of dict which including notes and beat. 导入midi文件，并将其转换为一个列表，这个列表里的元素是包含音符和节拍信息的字典。

In [ ]:
# 设定演奏者
avatar = 'asuka'
# 设定midi文件路径
midiFilePath = "asset/midi/Sunburst.mid"
# 设定blender里的FPS
FPS = 30
# 设定各弦音高
guitar_string_notes = ["d", "b", "G", "D", "A", "D1"]

filename = midiFilePath.split("/")[-1].split(".")[0]
left_hand_recorder_file = f"output/{filename}_lefthand_recorder.json"
left_hand_animation_file = f"output/{avatar}_{filename}_lefthand_animation.json"
right_hand_recorder_file = f"output/{filename}_righthand_recorder.json"
right_hand_animation_file = f"output/{avatar}_{filename}_righthand_animation.json"

tempo_changes, ticks_per_beat = get_tempo_changes(midiFilePath)
notes_map = midiToGuitarNotes(midiFilePath, useChannel=1)

In [ ]:
print(f'全曲的速度变化是:')
for track,tempo,tick in tempo_changes:
    print(f'在{track}轨，tick为{tick}时，速度变为{tempo}')
print(f'\n全曲的每拍tick数是:{ticks_per_beat}\n')
total_tick = notes_map[-1]['real_tick']
total_frame = calculate_frame(tempo_changes, ticks_per_beat, FPS,total_tick)
total_time = total_frame/FPS
print(f'如果以{FPS}的fps做成动画，一共是{total_tick} ticks, 合计{total_frame}帧, 约{total_time}秒')

## init guitar and handPoseRecordPool 初始化吉他和handPoseRecordPool
init guitar with a list of string. here we use a standard 6-string guitar. 用一个字符串列表来初始化吉他。这里我们使用标准的6弦吉他。
init the start hand pose. here we put a hand at barrel 0 and put fingers on string 3, which is G string. 初始化起始手型。在这里，我们将设定初始手型位置为第0品，并将手指放在第3弦上，也就是G弦上。

handPoseRecordPool can init with param `size`, which is the size of the pool. Bigger size will cost more memory and more time but can calculate more possibilities. handPoseRecordPool可以通过参数`size`初始化，即池的大小。越大就会消耗更多的内存和时间，但可以计算更多可能性。

In [ ]:

guitar_string_list = createGuitarStrings(guitar_string_notes)
# 初始化吉它
guitar = Guitar(guitar_string_list)
# 设定各手指状态
leftFingers = [
    LeftFinger(1, guitar_string_list[1], 1),
    LeftFinger(2, guitar_string_list[2], 2),
    LeftFinger(3, guitar_string_list[3], 3),
    LeftFinger(4, guitar_string_list[4], 4)
]
# 初始化左手
initLeftHand = LeftHand(leftFingers)
# 初始化第一个记录器
handPoseRecord = HandPoseRecorder()
handPoseRecord.addHandPose(initLeftHand, 0,0)
# 初始化记录池
handPoseRecordPool = HandPoseRecordPool(100)
handPoseRecordPool.insert_new_hand_pose_recorder(handPoseRecord,0)

current_recoreder_num = 0
previous_recoreder_num = 0

## Calculate the possibility of hand pose 计算手型的可能性
- For each notes, we calculate all possibile chords, which includes position infomations of notes on guitar. 对于每个音符，我们计算所有可能的和弦，包括音符在吉他上的位置信息。
- For each chords, we calculate all possibile hand poses, which includes the position of fingers on guitar.
- For each hand poses, we generate a LeftHand object if it is possible to play. 对于每个和弦，我们计算所有可能的手型，包括手指在吉他上的位置。
- For each LeftHand object, we calculate entropy, the cost of switch hand pose. Then we append it to the handPoseRecord. 对于每个手型，如果可以演奏，我们就生成一个LeftHand对象。
- For each handPoseRecord, we put it in the handPoseRecordPool if its entropy is lower than the record in the pool. 对于每个LeftHand对象，我们计算熵，即切换手型的成本。然后将其添加到handPoseRecord中。
- After all notes are calculated, we get the best handPoseRecord from the pool. Then print the result. 计算完所有音符后，我们从池中获取最佳handPoseRecord，然后打印结果


In [ ]:
from FretDaner import update_recorder_pool

total_steps = len(notes_map)
current_recoreder_num = 0
previous_recoreder_num = current_recoreder_num

update_recorder_pool(total_steps,guitar,handPoseRecordPool,notes_map, current_recoreder_num, previous_recoreder_num)


# after all iterations, read the best solution in the recorder pool. 全部遍历完以后，读取记录池中的最优解。
bestHandPoseRecord = handPoseRecordPool.curHandPoseRecordPool[0]
bestEntropy = bestHandPoseRecord.currentEntropy
print(f"最小消耗熵为：{bestEntropy}")
bestHandPoseRecord.output(True)
bestHandPoseRecord.save(left_hand_recorder_file,tempo_changes,ticks_per_beat,FPS)
print(f"总音符数应该为{total_steps}")
print(f"实际输出音符数为{len(bestHandPoseRecord.handPoseList)}")


## Initialize Animation Values 初始化动画所需数值

In Blender, by determining the positions of the following key points: IK pivot point, Y-axis rotation value, we can determine the entire left hand's posture. These key points are:
在blender中，通过确定以下几个关键点的位置，ik轴点，y轴旋转值，就可以确定整个左手的状态，这几个关键点分别是：
- Palm 手掌
- Thumb 大拇指
- Index finger 食指
- Middle finger 中指
- Ring finger 无名指
- Pinky finger 小指

There are four extreme positions of the hand shape, which are:
而手型有四个极端位置，分别是：
- Open string position on the 1st string 1弦空品
- Open string position on the 6th string 6弦空品
- 12th fret position on the 1st string 1弦12品
- 12th fret position on the 6th string 6弦12品

For each extreme position, we can adopt three methods to press the strings:
而每个极端位置，我们都可以采用三种方式来按弦，分别是：
- Index finger and middle finger on different frets, with the whole palm parallel to the neck 食指和中指在不同品上，整个手掌平行于琴颈
- Index finger and middle finger on the same fret, with the index finger pointing outward and the middle finger pointing inward, requiring the palm to rotate at a certain angle 食指与中指同品，而食指向外，中指向内，此时手掌要旋转一定角度
- Index finger and middle finger on the same fret, with the index finger pointing inward and the middle finger pointing outward, requiring the palm to rotate at a certain angle 食指与中指同品，而食指向内，中指向外，此时手掌要旋转一定角度

In Blender, we record the values of all key points for these four extreme positions in three different ways, totaling 12 hand shapes. 在blender中，我们记录下来以上四个极端位置的三种不同方式下，所有关键点的值，共计12种手型。

Through these 12 hand shapes, we can obtain the hand shape at any position through interpolation. 通过这12种手型，我们可以通过插值的方式，得到任意位置的手型。

In [ ]:
from src.animate.animate import leftHand2Animation
# 生成动画文件

leftHand2Animation(avatar,left_hand_recorder_file, left_hand_animation_file, tempo_changes, ticks_per_beat, FPS)

## Calculate the right-hand movements 计算右手的动作

Similar to the principle of calculating the left hand, our left hand starts from an initial hand shape. Then, each time a string needs to be strummed, we calculate the possible next strumming hand shapes and the entropy consumed by each hand shape. These hand shapes are recorded in a rightHandRecorderPool. Ultimately, through repeated iteration, the strumming sequence that consumes the least entropy is selected.

和计算左手的原理类似，我们的左手从一个初始手型开始，然后每一次需要拨弦时，就计算下一次可能有哪些拨弦手型，并且计算每个手型所消耗的熵，把这些手型都记录在一个rightHandRecorderPool里，最终反复迭代，选出消耗熵最小的拨弦序列。

In [ ]:
from src.hand.RightHand import RightHand
from src.HandPoseRecorder import RightHandRecorder,HandPoseRecordPool
initRightHand = RightHand(usedFingers=[],rightFingerPositions=[5,2,1,0])

initRightHandRecorder = RightHandRecorder()
initRightHandRecorder.addHandPose(initRightHand, 0,0)

rightHandRecordPool = HandPoseRecordPool(100)
rightHandRecordPool.insert_new_hand_pose_recorder(initRightHandRecorder,0)

In [ ]:
from FretDaner import update_right_hand_recorder_pool


update_right_hand_recorder_pool(
    left_hand_recorder_file, rightHandRecordPool, current_recoreder_num, previous_recoreder_num)

# after all iterations, read the best solution in the record pool. 全部遍历完以后，读取记录池中的最优解。
bestHandPoseRecord = rightHandRecordPool.curHandPoseRecordPool[0]
bestEntropy = bestHandPoseRecord.currentEntropy
print(f"最小消耗熵为：{bestEntropy}")
bestHandPoseRecord.output()
bestHandPoseRecord.save(right_hand_recorder_file,tempo_changes,ticks_per_beat,FPS)

In [ ]:
from src.animate.animate import rightHand2Animation

rightHand2Animation(avatar,right_hand_recorder_file,
                    right_hand_animation_file, tempo_changes, ticks_per_beat, FPS)